In [1]:
# imports

import os
import re
import json
from dotenv import load_dotenv
from huggingface_hub import login
from openai import OpenAI
from pricer.items  import Item
from pricer.evaluator import evaluate

In [2]:

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
dataset = "ed-donner/items_lite"
train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [4]:
openai = OpenAI()

# Data size

OpenAI recommends fine-tuning with a small population of 50-100 examples

so, i m goona go with 100 examples


In [5]:

fine_tune_train = train[:100]
fine_tune_validation = val[:50]

In [6]:
len(fine_tune_train)

100

# Step 1

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [7]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
        {"role": "assistant", "content": f"${item.price:.2f}"}
    ]

In [8]:
messages_for(fine_tune_train[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \nCategory: Home Hardware  \nBrand: Schlage  \nDescription: A single‑piece oil‑rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \nDetails: Designed for a 4" minimum center‑to‑center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation.'},
 {'role': 'assistant', 'content': '$64.30'}]

In [9]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [11]:
make_jsonl(fine_tune_train[:2])

'{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\\n\\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \\nCategory: Home Hardware  \\nBrand: Schlage  \\nDescription: A single\\u2011piece oil\\u2011rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \\nDetails: Designed for a 4\\" minimum center\\u2011to\\u2011center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation."}, {"role": "assistant", "content": "$64.30"}]}\n{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\\n\\nTitle: Mini Electric Air Duster Fan  \\nCategory: Electronics  \\nBrand: Kica  \\nDescription: Ultra\\u2011compact 86,000\\u202fRPM electric air duster with 11\\u202fm/s wind speed for precise cleaning and inflation.  \\nDetails: Powered by a 9.99\\u202fWh motor, adjustable in four speed

In [12]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [13]:
write_jsonl(fine_tune_train, "jsonl/fine_tune_train.jsonl")


In [20]:
write_jsonl(fine_tune_validation, "jsonl/fine_tune_validation.jsonl")


In [21]:
with open("jsonl/fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [22]:
train_file

FileObject(id='file-2YUvYiSPBe6gc56uZzWrdH', bytes=55120, created_at=1771412640, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [23]:
with open("jsonl/fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [24]:
validation_file

FileObject(id='file-UGAwTmsDLaViv7a9osRYmB', bytes=27637, created_at=1771412644, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

# Step 2

## And now time to Fine-tune!

In [16]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4.1-nano-2025-04-14",
    seed=42,
    hyperparameters={"n_epochs": 1, "batch_size": 1},
    suffix="pricer"
)

NameError: name 'validation_file' is not defined